In [10]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint

In [6]:
env_config = dotenv_values("../environment/.env")

In [8]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")

In [9]:
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

## Todos los resultados

In [39]:
params = {
    "index": "boe",
    "query": {"match_all": {}},
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [], 'max_score': None, 'total': {'relation': 'eq', 'value': 0}}


## Resultados semánticos con KNN

In [40]:
query_text = "cantidad de agua tope"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 2,
    "num_candidates": 5,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [], 'max_score': None, 'total': {'relation': 'eq', 'value': 0}}


## Combinado

In [47]:
query_text = "cantidad de agua tope"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
    "filter": {
        "match": {
            "text": "órgano composición",
        },
    },
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': '-OvttIsBir71ATlvNXOF',
           '_index': 'boe',
           '_score': 0.62262243,
           '_source': {'doc_id': 'BOE-A-2023-22766',
                       'text': '4.ª\u2003El folleto deberá declarar '
                               'expresamente aquellos casos en los que el '
                               'valor liquidativo de la IIC pueda presentar '
                               'una alta volatilidad debido a la composición '
                               'de su cartera o a las técnicas empleadas de '
                               'gestión de carteras.'}},
          {'_id': 'C-vttIsBir71ATlvN3T3',
           '_index': 'boe',
           '_score': 0.5943013,
           '_source': {'doc_id': 'BOE-A-2023-22766',
                       'text': 'Los informes semestral y, en su caso, '
                               'trimestral deberán contener información sobre '
                               'los aspectos indicados en el artículo '
                           